In [1]:
import json
import deepdoctection as dd

In [2]:
page = dd.load_page("deepdocdetect_output.json")

/Users/jhoetter/deepdoctection/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
scale_x = 819 / 2481 # png width / root level width
scale_y = 1060 / 3508

Categories:
- table
- title
- text
- cell

In [4]:
annotations = {
    "tables": [],
    "titles": [],
    "texts": []
}

In [5]:
for annotation in page.annotations:
    if str(annotation.category_name) == 'LayoutType.table':
        annotations["tables"].append(annotation)
    elif str(annotation.category_name) == 'LayoutType.title':
        annotations["titles"].append(annotation)
    elif str(annotation.category_name) == 'LayoutType.text':
        annotations["texts"].append(annotation)

In [6]:
for key, list in annotations.items():
    print(key, len(list))

tables 1
titles 2
texts 1


In [7]:
len(page.annotations)

410

In [8]:
table_example = page.annotations[0]

In [10]:
title_example = page.annotations[1]

In [11]:
text_example = page.annotations[2]

In [12]:
cell_example = page.annotations[5]

In [13]:
with open("deepdocdetect_output.json", "r") as f:
    content = json.load(f)

In [14]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [67]:
example = annotations["tables"][0]

In [124]:
def get_table_values(table):
    example_relationships = example.relationships[next(iter(example.relationships))]
    annotation_id_to_value = {}
    for idx, annotation in enumerate(page.get_annotation(annotation_ids=example_relationships)):
        if len(annotation.relationships) > 0:
            for cell_annotation in page.get_annotation(annotation_ids=annotation.relationships[next(iter(annotation.relationships))]):
                value = cell_annotation.sub_categories[next(iter(cell_annotation.sub_categories))].value
                annotation_id_to_value[cell_annotation.annotation_id] = value
    return annotation_id_to_value

{'9817c162-3e84-3cdb-876f-2d70c27cb32b': '15.63%',
 '6113d98b-780e-3276-8b20-07d5a0ba16b2': '$0.00',
 '67758b9d-2b64-3294-b59b-6cdbb5540c98': '$51,000.00',
 '1e7758bf-2bc2-3fa0-bda0-cedef26a7568': '$9,000.00',
 '548633c3-4231-346e-b2ff-810bb0510745': '8/1/2013',
 'c1c7f27c-597e-3f2d-a621-4a32d7f9346d': '$250.00',
 '6d83c355-86e8-3e70-ac79-7576056c7a45': 'HR',
 '02d9d56e-d5bf-33c2-a4fd-e52968a3d865': 'Remaining',
 '423a7461-769c-3762-a4f3-abb1bc3197c0': '$0.00',
 'bab5e683-3c10-3a73-a895-35d4f9355dc2': 'DF',
 '69bcd758-6e61-3271-a6e7-11b36a40d4a4': '$9,000.00',
 '8c405931-e73b-3138-b08b-088a6327aa9f': '$51',
 'd62ef1ae-c23a-3016-911a-3e9bbb170272': 'Schematic',
 'fe1aab6b-6db9-396a-95dc-0ef2c12b540c': 'Design',
 '5f302b9d-a3a6-360c-8858-932af4f978ac': 'GALT-009',
 'b1187bd7-b0b4-33c7-96da-bb835dd13796': 'Extension',
 '83a5227a-1971-3a45-aec1-d50b4179826c': 'Extension',
 'f5ec70e9-8bc6-35bf-8827-6ae1823d0bea': '$375.00',
 '9e7a48c1-9bab-32c4-aa5b-e0488ef31617': '$375.00',
 '9549db51-24c4

In [97]:
annotation_id_to_value

{'e11cb53c-3c07-32c4-89d7-56672089c0a1': '68.36%',
 '4ca7e8c7-bfba-31b1-a897-1ab513561304': '$350,000.00',
 '76c0aaef-11b1-35a9-a9bc-53f6ffe79f12': '82.86%',
 '487483fb-9797-3836-922c-da360dae00bf': '$290,000.00',
 '55c567ae-201a-30c8-b527-dd74c748ecc7': '2.57%'}

In [62]:
example_relationships = example.relationships[next(iter(example.relationships))]
annotation_id_to_value = {}
for annotation in page.get_annotation(annotation_ids=example_relationships):
    value = annotation.sub_categories[next(iter(annotation.sub_categories))].value
    annotation_id_to_value[annotation.annotation_id] = value
annotation_id_to_value

AttributeError: 'CategoryAnnotation' object has no attribute 'value'

In [45]:
len(page.get_annotation(annotation_ids=example_relationships))

94

In [ ]:


img = plt.imread("examples/invoice-1.png")
fig, ax = plt.subplots(1, figsize=(15, 15))

# Display the image
ax.imshow(img)

rows = [{
    'ulx': example.bounding_box.ulx * scale_x,
    'uly': example.bounding_box.uly * scale_y,
    'lrx': example.bounding_box.lrx * scale_x,
    'lry': example.bounding_box.lry* scale_y,
    "content": "wow"
}]
for box in rows:
    print(box['uly'], box['lry'])
    # rect = patches.Rectangle((box['position_x0'], box['position_y0']), box['position_x1']-box['position_x0'], box['position_y1']-box['position_y0'], linewidth=1, edgecolor='r', facecolor='none')
    rect = patches.Rectangle((box['ulx'], box['uly']), box['lrx']-box['ulx'], box['lry']-box['uly'], linewidth=1, edgecolor='r', facecolor='none')
    print(rect)
    ax.add_patch(rect)
    #ax.annotate(box['content'], (box['ulx'], box['uly']), fontsize=8, color='r')

# Show the image
plt.show()